In [1]:
import pandas as pd
import pypistats
import json
import numpy as np
import pyvis
import palettable as palette
from pyvis.network import Network


# Generate data

In [2]:
# Load edges             
from ecosystem_edges import edges
edges = pd.DataFrame(edges)

In [3]:
# Extract node list and assign URLs to PyPi
pkgs = list(set(edges["from"].to_list() + edges["to"].to_list()))
node_list = [{"id": pkg, "category": None, "url": f"https://pypi.org/project/{pkg}/"} for pkg in pkgs]
nodes = pd.DataFrame(node_list)

In [5]:
# Assign categories
# ------------------

# Load categories
from ecosystem_categories import categories

# Update categories
for category, data in categories.items():
    nodes.loc[nodes["id"].isin(data["pkgs"]), "category"] = category
    
nodes["downloads"] = None
    
# Update downloads
for idx, row in nodes.iterrows():
    r = pypistats.recent(row["id"], "month", format="json")
    downloads = json.loads(r)["data"]["last_month"]
    nodes.loc[idx, "downloads"] = downloads
    
nodes["log10_downloads"] = np.log10(nodes["downloads"].astype(float))
nodes["log2_downloads"] = np.log2(nodes["downloads"].astype(float))

# Visualize with Pyvis

In [ ]:
# COLORS
color_palette = palette.wesanderson.GrandBudapest5_5.hex_colors

options = """
var options = {
    "nodes": {
        "font": {
            "multi": "html",
            "bold": "16px arial black"
        }
    }
}
"""

def get_color(category):
    # Color scheme
    colors = {"analysis / modelling": color_palette[0],
              "core / data structures": color_palette[1],
              "visualization": color_palette[2],
              "data extraction / processing": color_palette[3]}
    return colors[category]

def get_node_color(nodes, name):
    try:
        return get_color(nodes.loc[nodes["id"]==name]["category"].values[0])
    except Exception as e:
        print(name)
        

def get_node_size(nodes, name, size_column):
    return nodes.loc[nodes["id"]==name][size_column].values[0]


#nt = Network('1000px', '1500px', notebook=True, bgcolor='#222222', font_color='white')
nt = Network('1000px', '1500px', notebook=True)


# Add nodes and edges
for idx, edge in edges.iterrows():
    source = edge["from"]
    target = edge["to"]
    nt.add_node(source, source, title=f"<b>{source}</b>", size=get_node_size(nodes, source, "log2_downloads"), color=get_node_color(nodes, name=source), font={"size":16})
    nt.add_node(target, target, title=f"<b>{source}</b>", size=get_node_size(nodes, target, "log2_downloads"), color=get_node_color(nodes, name=target), font={"size":16})
    if source == "pysal" and target == "geopandas":
        nt.add_edge(source, target, weight=2, color="grey")
    else:
        nt.add_edge(source, target, weight=2, color="grey")

# Add Legend Nodes
step = 50
x = -1000
y = 200

for i, key in enumerate(categories.keys()):
    nt.add_node(n_id=i, group=key, label=key, size=80, physics=False, x=x, y=f"{y + i*step}px", shape="box", widthConstraint=200, font={"size":16, "color": "white"},
               color=get_color(key)) 



In [ ]:

#nt.force_atlas_2based(gravity=100, central_gravity=0.9, spring_length=200)
nt.set_options(options)
#nt.show_buttons(filter_=['physics'])
nt.show("Python-ecosystem.html")
